# Incident Report from video

In [1]:
import os
import cv2
import math
import numpy as np
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

In [2]:
video_file = "data/insurance_v3.mp4"
number_of_frames = 14

# Create the directory to store the frames
os.makedirs("data/frames", exist_ok=True)

# Extract the frames from the video
video = cv2.VideoCapture(video_file)
frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    frames.append(buffer)
video.release()

# Save the frames
for i, frame in enumerate(frames[::math.ceil(len(frames) / number_of_frames)]):
    image = cv2.imdecode(np.frombuffer(frame, np.uint8), cv2.IMREAD_COLOR)
    cv2.imwrite(f"data/frames/frame_{i}.jpg", image)

In [6]:
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini" #"gpt-4o-mini"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage (
            content=[
                TextContentItem(text="""
                    You are an expert in evaluating car damage from car accidents for auto insurance reporting. 
                    Create an incident report for the accident shown in the video with 3 sections. 

                    Section 1 will include the car details (license plate, car make, car model, approximant model year, color, mileage).
                    Section 2 list the car damage, per damage in a list 
                    Section 3 will only include exactly 6 sentence description of the car damage.
                """),
                *[ImageContentItem(image_url=ImageUrl.load(image_file=os.path.join("data/frames", filename), 
                                                           image_format="jpg", 
                                                           detail=ImageDetailLevel.LOW)) 
                    for filename in os.listdir("data/frames")]
            ]
        ),
    ],
    model=model_name,
    temperature=1.,
    max_tokens=4096,
    top_p=1.
)

print(response.choices[0].message.content)

### Incident Report

#### Section 1: Car Details
- **License Plate:** MAT 251
- **Car Make:** Toyota
- **Car Model:** Camry
- **Approximate Model Year:** 2007
- **Color:** Blue
- **Mileage:** 151,856

#### Section 2: Car Damage
1. Scratches along the left side door
2. Damage to the rear bumper with scratches
3. Dents on the rear quarter panel
4. Scratches near the fuel door
5. Scratch and dent on the rear trunk lid
6. Minor scratches on the front hood

#### Section 3: Damage Description
The car exhibits multiple scratches along the left-side door, indicating a side impact. The rear bumper has noticeable damage with extensive scratches and some dents. Additionally, there are multiple scratches located near the fuel door and on the rear quarter panel. The trunk lid shows evidence of a dent along with scratches. The front hood has minor scratches, contributing to the overall appearance of wear. Overall, the vehicle shows signs of significant wear from the collision.
